In [4]:
import pysolr
from __future__ import print_function
import tweepy
import json
import csv

In [7]:
credentials = {}
credentials['api_key'] = 'n2dTtVbZuJwji0auruBGVTBwT'
credentials['api_secret_key'] = 'U1Er7hNUFtOwmPIz6BCr6w1B3o9GlKhg6loffJVIK0cFAFdkYl'
credentials['access_token'] = '987093510116532231-tiHjc16Ws3vcMZpNUt0kuL7xj8iLSy1'
credentials['access_token_secret'] = 'ykZmkTz2TIeSajyWDKXvSSxAiFZvMFdD9ePUe0zn5fsna'

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)

# Load Twitter API secrets from an external JSON file
api_key = credentials['api_key']
api_secret_key = credentials['api_secret_key']
access_token = credentials['access_token']
access_token_secret = credentials['access_token_secret']

# Twitter initialization
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [6]:
# Create a client instance. The timeout and authentication options are not required.
#solr = pysolr.Solr('http://localhost:8983/solr/', always_commit=True, timeout=10)

# Note that auto_commit defaults to False for performance. You can set
# `auto_commit=True` to have commands always update the index immediately, make
# an update call with `commit=True`, or use Solr's `autoCommit` / `commitWithin`
# to have your data be committed following a particular policy.

#*******************************************************************************
# For SolrCloud mode, initialize your Solr like this:
zookeeper = pysolr.ZooKeeper("localhost:8983")
solr = pysolr.SolrCloud(zookeeper, "Tweets")

# Do a health check.
solr.ping()

Connection dropped: socket connection broken
Connection dropped: socket connection broken
Connection dropped: socket connection broken
Connection dropped: socket connection broken
Connection dropped: socket connection broken
Connection dropped: socket connection broken
Connection dropped: socket connection broken
Connection dropped: socket connection broken
Connection dropped: socket connection broken
Connection dropped: socket connection broken
Connection dropped: socket connection broken
Connection dropped: socket connection broken
Connection dropped: socket connection broken
Connection dropped: socket connection broken
Connection dropped: socket connection broken
Connection dropped: socket connection broken
Failed connecting to Zookeeper within the connection retry policy.


KazooTimeoutError: Connection time-out

In [ ]:
# How you'd index data.
solr.add([
    {
        "id": "doc_1",
        "title": "A test document",
    },
    {
        "id": "doc_2",
        "title": "The Banana: Tasty or Dangerous?",
        "_doc": [
            { "id": "child_doc_1", "title": "peel" },
            { "id": "child_doc_2", "title": "seed" },
        ]
    },
])

# You can index a parent/child document relationship by
# associating a list of child documents with the special key '_doc'. This
# is helpful for queries that join together conditions on children and parent
# documents.

In [ ]:
# Later, searching is easy. In the simple case, just a plain Lucene-style
# query is fine.
results = solr.search('bananas')

# The ``Results`` object stores total results found, by default the top
# ten most relevant results and any additional data like
# facets/highlighting/spelling/etc.
print("Saw {0} result(s).".format(len(results)))

# Just loop over it to access the results.
for result in results:
    print("The title is '{0}'.".format(result['title']))

# For a more advanced query, say involving highlighting, you can pass
# additional options to Solr.
results = solr.search('bananas', **{
    'hl': 'true',
    'hl.fragsize': 10,
})

# You can also perform More Like This searches, if your Solr is configured
# correctly.
similar = solr.more_like_this(q='id:doc_2', mltfl='text')

In [ ]:
# Finally, you can delete either individual documents,
solr.delete(id='doc_1')

# also in batches...
solr.delete(id=['doc_1', 'doc_2'])

# ...or all documents.
solr.delete(q='*:*')